In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Text input to top N closest songs

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import pandas as pd
import joblib
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_VAD(sentiment_dimension, text_inputs, dir):
  model_path = f'{dir}/{sentiment_dimension}_model'
  model = AutoModelForSequenceClassification.from_pretrained(model_path)
  model.to(device)

  tokenizer = AutoTokenizer.from_pretrained(model_path)

  inputs = tokenizer(text_inputs, return_tensors="pt", padding=True, truncation=True)
  inputs.to(device)

  outputs = model(**inputs)

  predictions = outputs.logits.squeeze().detach().cpu().numpy()

  return predictions

def predict_audio_features(input):
  VAD_to_features = joblib.load(f'{dir}/VAD_to_audiofeatures.joblib')

  scaler_X = joblib.load(f'{dir}/scaler_X.joblib')
  scaler_y = joblib.load(f'{dir}/scaler_y.joblib')

  input_sentiment_scaled = scaler_X.transform(input)

  predicted_audio_features_scaled = VAD_to_features.predict(input_sentiment_scaled)
  predicted_audio_features = scaler_y.inverse_transform(predicted_audio_features_scaled)

  return predicted_audio_features

def text_to_songs(dir, input, num_songs, df_orig):
  valence_pred = predict_VAD('valence', input, dir)
  arousal_pred = predict_VAD('arousal', input, dir)
  dominance_pred = predict_VAD('dominance', input, dir)

  scaled_valence = valence_pred * 2 - 1
  scaled_arousal = arousal_pred * 2 - 1
  scaled_dominance = dominance_pred * 2 - 1

  input_sentiment = pd.DataFrame({
      'valence_tags': [scaled_valence],
      'arousal_tags': [scaled_arousal],
      'dominance_tags': [scaled_dominance]
  })

  predicted_audio_features = predict_audio_features(input_sentiment)
  print(predicted_audio_features)

  similarities = cosine_similarity(predicted_audio_features, df_orig[['danceability', 'energy', 'loudness', 'valence', 'tempo']])
  similarities = similarities[0]

  closest_songs_indices = np.argsort(similarities)[::-1][:num_songs]

  df_orig_reset = df_orig.reset_index(drop=True)
  closest_songs = df_orig_reset.loc[closest_songs_indices]
  print(closest_songs)


In [14]:
# Change these as necessary

# Directory containing models and scalers
# dir = os.getcwd()
dir = '/content/gdrive/MyDrive/Colab Notebooks'

# Number of songs to find
num_songs = 10

# Dataframe with songs to pull from to generate playlist
df = pd.read_csv(f'{dir}/lyrics_with_audio_47k.csv')
df = df.dropna()

# Input sentence
input = "I am extremely sad because my dog died"

text_to_songs(dir, input, num_songs, df)

[[  0.4092145    0.86790308  -7.77298979   0.39669808 128.01671366]]
       Unnamed: 0                                         lastfm_url  \
2235       2667.0  https://www.last.fm/music/lucybell/_/cuando%2b...   
4346       5408.0  https://www.last.fm/music/beat%2bhappening/_/b...   
24419     30128.0  https://www.last.fm/music/of%2bmonsters%2band%...   
29388     36214.0  https://www.last.fm/music/klee/_/f%25c3%25bcr%...   
17243     21479.0  https://www.last.fm/music/decoded%2bfeedback/_...   
18232     22761.0  https://www.last.fm/music/sarah%2bbrightman/_/...   
20671     25589.0  https://www.last.fm/music/vain/_/down%2bfor%2b...   
17494     21816.0  https://www.last.fm/music/beulah/_/sunday%2bun...   
14824     18557.0         https://www.last.fm/music/weezer/_/holiday   
1074       1194.0  https://www.last.fm/music/boysetsfire/_/the%2b...   

                           track               artist                   seeds  \
2235   Cuando Respiro en Tu Boca             Lucybell    

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MLPRegressor was fitted with feature names
  warnings.warn(
